In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import models, layers, optimizers
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re

%matplotlib inline

import os
print(os.listdir("../input"))


['amazonreviews']


In [36]:
import keras
from sklearn.model_selection import train_test_split
from keras import layers
from keras.layers.core import Activation, Dense
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential

Data proprecessing

In [2]:
#Read the text
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels[:10000]), texts[:10000]
train_labels, train_texts = get_labels_and_texts('../input/amazonreviews/train.ft.txt.bz2')
test_labels, test_texts = get_labels_and_texts('../input/amazonreviews/test.ft.txt.bz2')

In [7]:
import re
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        
train_texts = normalize_texts(train_texts)
test_texts = normalize_texts(test_texts)

In [11]:
import re
token=re.compile('[A-Za-z]+| [!?.,()\']')
def reg_text(text):
    
    new_text=token.findall(text)
    
    new_text=[word.lower() for word in new_text]
    return new_text

In [31]:
def vector_generator(x):
    
    #x = pd.DataFrame(x,columns=['text'])
    x['text']=x.text.apply(reg_text)
    word_set=set()
    for text in x.text:
        for word in text:
            word_set.add(word)
    word_list=list(word_set)  
   

    word_index=dict( (word,word_list.index(word)) for word in word_list )
    data_input=x.text.apply(lambda x: [word_index.get(word, 0) for word in x])
    
    maxlen=max(len(x)for x in data_input)
    maxword=len(word_list)+1
    
    data_input=keras.preprocessing.sequence.pad_sequences(data_input.values,maxlen=maxlen)
    
    return data_input, maxlen, maxword
    

In [60]:
train_x = pd.DataFrame(train_texts,columns=['text'])
test_x  = pd.DataFrame(test_texts,columns=['text'])


data=pd.concat([train_x,test_x]) 
data.head()

,text
0,stuning even for the non gamer this sound tra...
1,the best soundtrack ever to anything i m rea...
2,amazing this soundtrack is my favorite music...
3,excellent soundtrack i truly like this soundt...
4,remember pull your jaw off the floor after he...


In [66]:
#Generate train set and text set
data, maxlen, maxword =vector_generator(data)

data_input=data[:10000]
data_test=data[10000:]

In [67]:
# generate training set and test set
Xtrain, Xtest, ytrain, ytest = train_test_split(data_input, train_labels, test_size=0.2, random_state=42)

Build model  and Train model

In [68]:
# Build LSTM model
model = Sequential()
model.add(layers.Embedding(maxword,1024,input_length=maxlen))

model.add(layers.LSTM(1024,dropout=0.5, recurrent_dropout=0.5))


#model.add( layers.Dense(522,kernel_regularizer=regularizers.l2(0.1),activation='relu') )
#model.add( layers.Dropout(0.5) )



model.add(Dense(1,activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

In [69]:
model.fit(Xtrain, ytrain, batch_size=200, epochs=12,validation_data=(Xtest, ytest))

/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8000 samples, validate on 2000 samples
Epoch 1/12
8000/8000 [==============================] - 24s 3ms/step - loss: 0.6444 - accuracy: 0.6315 - val_loss: 0.5409 - val_accuracy: 0.7085
Epoch 2/12
8000/8000 [==============================] - 23s 3ms/step - loss: 0.4323 - accuracy: 0.8150 - val_loss: 0.3785 - val_accuracy: 0.8455
Epoch 3/12
8000/8000 [==============================] - 23s 3ms/step - loss: 0.2936 - accuracy: 0.8859 - val_loss: 0.5047 - val_accuracy: 0.7520
Epoch 4/12
8000/8000 [==============================] - 23s 3ms/step - loss: 0.2178 - accuracy: 0.9204 - val_loss: 0.3987 - val_accuracy: 0.8405
Epoch 5/12
8000/8000 [==============================] - 23s 3ms/step - loss: 0.1536 - accuracy: 0.9464 - val_loss: 0.6469 - val_accuracy: 0.8240
Epoch 6/12
8000/8000 [==============================] - 23s 3ms/step - loss: 0.1441 - accuracy: 0.9495 - val_loss: 0.5512 - val_accuracy: 0.8360
Epoch 7/12
8000/8000 [==============================] - 23s 3ms/step - loss: 0.108

Prodiction and evaluation

In [92]:
pred = model.predict(data_test)


pred_ = [1 if x > 0.5 else 0 for x in pred]

In [93]:
print(roc_auc_score(test_labels, pred_))

0.7960350218886804
